In [ ]:
source("Main.R")
source("Conf.R")
source("Utilities.R")
library("factoextra")
library("maptree")
library("ggpubr")

library(repr)
fileDir = "/home/beraslan/jovian-work/analysisSingle/"
geneCutOff=4
targetCutOff=14

In [ ]:
coefsAll_Control <- read.csv("./../ControlGuideEffects/Control_coefs.csv", row.names = 1)
pValsAll_Control <- read.csv("./../ControlGuideEffects/Control_pValues.csv", row.names = 1)

coefsAll <- read.csv("./../MixedEffectLMOutputs/ME_LMBetaCoefsALL.csv", row.names = 1)
pValsAll <- read.csv("./../MixedEffectLMOutputs/ME_LMPValuesALL.csv", row.names = 1)
pValsAll <- pValsAll[rownames(pValsAll) %ni% c("Intercept", "n_genes", "mt_frac", "Group Var"),]
coefsAll <- coefsAll[rownames(coefsAll) %ni% c("Intercept", "n_genes", "mt_frac", "Group Var"),]

In [ ]:
pValsAll = data.frame((pValsAll))
pValsAll_Control = data.frame((pValsAll_Control))

In [ ]:
pValsAllAdj <- sapply(pValsAll, 
                      function(x){p.adjust(x, method = "fdr", n = length(x))})
rownames(pValsAllAdj) <- rownames(pValsAll)

pValsAllAdj_Control <- data.frame(sapply(pValsAll_Control, 
                                         function(x){p.adjust(x, method = "fdr", n = length(x))}))
rownames(pValsAllAdj_Control) <- rownames(pValsAllAdj_Control)


In [ ]:
s1 <- apply(pValsAllAdj, 1, function(x){return(length(which(x < 0.1)))})

s1Freq <- data.frame(table(s1), stringsAsFactors = FALSE)

s1_Control <- apply(pValsAllAdj_Control, 1, 
                    function(x){return(length(which(x < 0.1)))})

In [ ]:
s1Freq <- data.frame(table(s1), stringsAsFactors = FALSE)
s1Freq$s1 = as.numeric(s1Freq$s1)
head(s1Freq)
s1Freq$s1 <- as.character(s1Freq$s1)
s1Freq$s1 <- factor(as.character(s1Freq$s1), levels=as.character(sort(as.numeric(s1Freq$s1))))
 
options(repr.plot.width=10, repr.plot.height=5)
ggplot(data=s1Freq, aes(x=s1, y=Freq)) +
    geom_bar(stat="identity", fill="lightblue")+
    theme_minimal()+theme(
                     axis.text.y = element_text(size=18), 
                     axis.title = element_text(size=20) )+
    xlab("Number of affected genes")+ylab("Number of KO genes")+
geom_vline(xintercept = (targetCutOff), color="red")
scale_x_continuous(breaks=seq(2,1000,5), labels=seq(2,1000,5))
  

In [ ]:
tpKO <- data.frame(x=s1, type="KO")
tpC <- data.frame(x=s1_Control, type="Control")

tpKO <- rbind(tpKO, tpC)
tpKO$type <- factor(tpKO$type, levels=c("Control","KO"))

In [ ]:
options(repr.plot.width=8, repr.plot.height=4)

#pdf("./../SupplementaryFigures/S_3A.pdf", width = 8, height = 4)
ggplot(data=tpKO, aes(x=x, color=type, fill=type)) +
    geom_histogram(binwidth=1, alpha=0.1)+
    facet_wrap(~type)+
    theme_minimal()+theme(axis.text.x = element_text(size=18),
                     axis.text = element_text(size=18),
                     strip.text = element_text(size=18),
                     axis.title = element_text(size=20) )+
    xlab("Number of affected genes")+ylab("Number of perturbations")+
geom_vline(xintercept = (targetCutOff), color="red")+xlim(0,300)+
scale_fill_manual(values=c("blue", "#E69F00", "#56B4E9"))+
scale_color_manual(values=c("blue", "#E69F00", "#56B4E9"))
#dev.off()

In [ ]:
coefsAll <- coefsAll[s1>targetCutOff,]
pValsAllAdj <- pValsAllAdj[s1>targetCutOff,]

In [ ]:
s2 <- apply(pValsAllAdj, 2, function(x){return(length(which(x < 0.1)))})
s2Freq <- data.frame(table(s2), stringsAsFactors = FALSE)

In [ ]:
s2 <- apply(pValsAllAdj, 2, function(x){return(length(which(x < 0.1)))})
s2Freq <- data.frame(table(s2), 
                     stringsAsFactors = FALSE)
#s2Freq$s2 <- as.character(s2Freq$s2)
s2Freq$s2 <- factor(as.character(as.numeric(s2Freq$s2)), 
                    levels=as.character(sort(as.numeric(s2Freq$s2))))


In [ ]:
 
options(repr.plot.width=15, repr.plot.height=5)

pdf("./../SupplementaryFigures/S_3B.pdf", width = 15, height = 5)
ggplot(data=s2Freq, aes(x=s2, y=Freq)) +
    geom_bar(stat="identity", fill="lightblue")+
    theme_bw()+theme(axis.text.x = element_text(angle = 90, hjust = 1, size=12), axis.text.y = element_text(size=18), 
                     axis.title = element_text(size=20) )+scale_y_continuous(trans="log2")+
    ylab("Number of genes")+xlab("Number of KOs the gene is affected from")+geom_vline(xintercept = (geneCutOff+1), 
                                                                                       color="red", lwd=2)

dev.off()
  